In [47]:
import os
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import utils.genre_processing as gp
import utils.file_handler as fh
import utils.age_processing as ap
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_rel
from scipy.spatial.distance import jensenshannon

In [48]:
models = ['Random', 'MostPop', 'RP3beta', 'iALS']
dataset = 'mlhd'
ages_sort = ap.get_sorted_ages(dataset, 'defined_ages')
child_group = ages_sort[0]
mainstream_group = ages_sort[1]

In [49]:
results_path = f'../Results/{dataset}/user_and_recommendation_genre_distributions.tsv'
genres = gp.get_genres(dataset)

In [50]:
results = pd.read_csv(results_path, sep='\t')

C:\Users\rungruh\AppData\Local\Temp\ipykernel_9264\943527664.py:1: DtypeWarning: Columns (27,39,51,63) have mixed types. Specify dtype option on import or set low_memory=False.
  results = pd.read_csv(results_path, sep='\t')


In [51]:
for column in results.columns:
    if 'genre_distribution' in column:
        results[column] = results[column].apply(fh.safe_literal_eval)

In [52]:
results.columns

Index(['user_id', 'age', 'age_group', 'train_genre_distribution',
       'train_interactions', 'train_avg_popularity',
       'train_avg_normalized_popularity', 'train_avg_child_popularity',
       'train_avg_child_normalized_popularity',
       'validation_genre_distribution', 'validation_interactions',
       'validation_avg_popularity', 'validation_avg_normalized_popularity',
       'validation_avg_child_popularity',
       'validation_avg_child_normalized_popularity', 'test_genre_distribution',
       'test_interactions', 'test_avg_popularity',
       'test_avg_normalized_popularity', 'test_avg_child_popularity',
       'test_avg_child_normalized_popularity', 'MostPop_genre_distribution',
       'MostPop_interactions', 'MostPop_avg_popularity',
       'MostPop_avg_normalized_popularity', 'MostPop_avg_child_popularity',
       'MostPop_avg_child_normalized_popularity',
       'child_MostPop_genre_distribution', 'child_MostPop_interactions',
       'child_MostPop_avg_popularity',
   

In [53]:
def agp_deviation(df, models):
    
    mainstream_distribution = df[df['age_group'] == mainstream_group]['train_genre_distribution'].apply(lambda x: gp.genre_dict_to_list(x, dataset)).apply(np.array).mean(axis=0)
    child_distribution = df[df['age_group'] == child_group]['train_genre_distribution'].apply(lambda x: gp.genre_dict_to_list(x, dataset)).apply(np.array).mean(axis=0)

    df =  df[['user_id','train_genre_distribution', 'age_group'] + [f'child_{model}_genre_distribution' for model in models] + [f'{model}_genre_distribution' for model in models]]

    js_results_df = pd.DataFrame()

    for age_group, group in df.groupby('age_group'):

        js_df = pd.DataFrame()
        for model in models:
            js_df[f'js_mainstream_{model}'] = None
            js_df[f'js_child_{model}'] = None
            js_df[f'js_childsubset_mainstream_{model}'] = None
            js_df[f'js_childsubset_child_{model}'] = None
        js_df['user_id'] = None
        
        for i, row in group.iterrows():
            for model in models:
                recommendation_distribution = np.array(gp.genre_dict_to_list(row[f'{model}_genre_distribution'], dataset), dtype=float)
                js_mainstream = jensenshannon(recommendation_distribution, mainstream_distribution)**2
                js_child = jensenshannon(recommendation_distribution, child_distribution)**2

                if np.isnan(js_mainstream):
                    js_mainstream = 0
                if np.isnan(js_child):
                    js_child = 0
                js_df.at[i, f'js_mainstream_{model}'] = js_mainstream
                js_df.at[i, f'js_child_{model}'] = js_child
 
            js_df.at[i, 'user_id'] = row['user_id']
            js_df['age_group'] = age_group
            
        
        js_results_df = pd.concat([js_results_df, js_df], axis=0)               

        if age_group == child_group:

            js_df = pd.DataFrame()
            for model in models:
                js_df[f'js_mainstream_{model}'] = None
                js_df[f'js_child_{model}'] = None
                js_df[f'js_childsubset_mainstream_{model}'] = None
                js_df[f'js_childsubset_child_{model}'] = None
            js_df['user_id'] = None
            
            for i, row in group.iterrows():
                for model in models:
                    recommendation_distribution = np.array(gp.genre_dict_to_list(row[f'child_{model}_genre_distribution'], dataset), dtype=float)
                    js_mainstream = jensenshannon(recommendation_distribution, mainstream_distribution)**2
                    js_child = jensenshannon(recommendation_distribution, child_distribution)**2

                    if np.isnan(js_mainstream):
                        js_mainstream = 0
                    if np.isnan(js_child):
                        js_child = 0
                    js_df.at[i, f'js_mainstream_{model}'] = js_mainstream
                    js_df.at[i, f'js_child_{model}'] = js_child
    
                js_df.at[i, 'user_id'] = row['user_id']
                js_df['age_group'] = 'child_focused'
                
            
            js_results_df = pd.concat([js_results_df, js_df], axis=0)  

    return js_results_df


In [54]:
js_results_df = agp_deviation(results, models)

In [55]:
def test_agp_across_groups(js_df, models):
    for comparison in ['mainstream', 'child']:
        print('=====================================')
        print()
        print(f"Testing Deviation to {comparison} across age groups")
        print('=====================================')
        print()
        for model in models:
            print('--------------------------------------')
            print(f"Testing JSD to {comparison} for model: {model}")
            
            # Group JSD by age group for the given model
            js_df_clean = js_df[['age_group', f'js_{comparison}_{model}']].dropna()
            js_df_clean = js_df_clean[js_df_clean['age_group'] != 'child_focused']
            groups = [group[f'js_{comparison}_{model}'].values for name, group in js_df_clean.groupby('age_group')]
            
            # Print average scores
            print(f"Average JS divergence for {model}:")
            
            for age in ages_sort:
                print(f"Age group: {age}, Average JSD: {js_df_clean[js_df_clean['age_group'] == age][f'js_{comparison}_{model}'].mean():.4f}")

            # One-way ANOVA test
            anova_result = f_oneway(*groups)
            print(f"ANOVA result for {model}: F-statistic = {anova_result.statistic:.4f}, p-value = {anova_result.pvalue:.4f}")
            
            # If ANOVA is significant, perform Tukey HSD for post-hoc analysis
            if anova_result.pvalue < 0.05:
                print(f"ANOVA is significant for {model}, performing Tukey HSD test...")
                
                tukey_result = pairwise_tukeyhsd(pd.to_numeric(js_df_clean[f'js_{comparison}_{model}']), js_df_clean['age_group'], alpha=0.01)
                print(tukey_result)
            print('--------------------------------------')
            print()
            # paired t-test between age_group = child_focused and age_group = child
            print('Paired t-test between child_focused and child')
            child_focused = js_df[js_df['age_group'] == 'child_focused'][f'js_{comparison}_{model}'].astype(float)
            child = js_df[js_df['age_group'] == child_group][f'js_{comparison}_{model}'].astype(float)
            print(f'Mean child_focused: {child_focused.mean():.4f}')
            ttest_result = ttest_rel(child_focused, child)
            print(f"Paired t-test result: t-statistic = {ttest_result.statistic:.4f}, p-value = {ttest_result.pvalue:.4f}")
                
            print()
            print()
            print()

    

In [56]:
test_agp_across_groups(js_results_df, models)


Testing Deviation to mainstream across age groups

--------------------------------------
Testing JSD to mainstream for model: Random
Average JS divergence for Random:
Age group: 12-16, Average JSD: 0.0238
Age group: 17-29, Average JSD: 0.0239
Age group: 30-65, Average JSD: 0.0239
ANOVA result for Random: F-statistic = 0.1815, p-value = 0.8340
--------------------------------------

Paired t-test between child_focused and child
Mean child_focused: 0.0242
Paired t-test result: t-statistic = 1.5550, p-value = 0.1201



--------------------------------------
Testing JSD to mainstream for model: MostPop
Average JS divergence for MostPop:
Age group: 12-16, Average JSD: 0.0669
Age group: 17-29, Average JSD: 0.0672
Age group: 30-65, Average JSD: 0.0689
ANOVA result for MostPop: F-statistic = 14.5056, p-value = 0.0000
ANOVA is significant for MostPop, performing Tukey HSD test...
Multiple Comparison of Means - Tukey HSD, FWER=0.01
group1 group2 meandiff p-adj   lower  upper  reject
----------